<a href="https://colab.research.google.com/github/Ygarkakoito/neuron/blob/main/blank__05_egorova_eva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## 1. Представление и предобработка текстовых данных

1.1 Операции по предобработке:
* токенизация
* стемминг / лемматизация
* удаление стоп-слов
* удаление пунктуации
* приведение к нижнему регистру
* любые другие операции над текстом

In [3]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer

In [4]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

Реализовать функцию `preprocess_text(text: str)`, которая:
* приводит строку к нижнему регистру
* заменяет все символы, кроме a-z, A-Z и знаков .,!? на пробел


In [5]:
from nltk.tokenize import word_tokenize
import re

def preprocess_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^a-zA-Z.,!?]', ' ', text)
    return text

text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'
processed_text = preprocess_text(text)
print(processed_text)


select your preferences and run the install command. stable represents the most currently tested and supported version of pytorch. note that libtorch is only available for c  


1.2 Представление текстовых данных при помощи бинарного кодирования


Представить первое предложение из `text` в виде тензора `sentence_t`: `sentence_t[i] == 1`, если __слово__ с индексом `i` присуствует в предложении.

In [7]:
import torch
from nltk.tokenize import word_tokenize
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'
tokens = word_tokenize(text)
unique_words = sorted(set(tokens))
sentence_t = torch.zeros(len(unique_words))
for i, word in enumerate(unique_words):
    if word in tokens:
        sentence_t[i] = 1
print("Тензор предложения:",sentence_t)


Тензор предложения: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.])


## 2. Классификация фамилий по национальности

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`.

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)

2.5 Реализовать класс `SurnamesDataset`

2.6. Обучить классификатор.

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch
from torch.nn.utils.rnn import pad_sequence


In [27]:
# Шаг 2.1: Считываем файл
file_path = 'surnames/surnames.csv'
df = pd.read_csv(file_path)

In [28]:
# Шаг 2.2: Кодируем национальности числами
nationalities = df['nationality'].unique()
nationality_to_idx = {nat: idx for idx, nat in enumerate(nationalities)}
idx_to_nationality = {idx: nat for idx, nat in enumerate(nationalities)}
df['nationality'] = df['nationality'].map(nationality_to_idx)

In [29]:
# Шаг 2.3: Разбиваем датасет на обучающую и тестовую выборку
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [30]:
# Шаг 2.4: Реализуем класс Vocab
class Vocab:
    def __init__(self, data):
        self.idx_to_token = {i: char for i, char in enumerate(data)}
        self.token_to_idx = {char: i for i, char in enumerate(data)}
        self.vocab_len = len(data)

In [36]:
class SurnamesDataset(Dataset):
    def __init__(self, X, y, vocab: Vocab):
        self.X = X
        self.y = y
        self.vocab = vocab

    def vectorize(self, surname):
        '''Генерирует представление фамилии surname в при помощи бинарного кодирования'''
        vectorized_surname = [self.vocab.token_to_idx[char] for char in surname]
        return torch.tensor(vectorized_surname)

    def collate_fn(self, batch):
            surnames = [item['surname'] for item in batch]
            nationalities = [item['nationality'] for item in batch]

            # Добавляем пробел в словарь, если его там нет
            if ' ' not in self.vocab.token_to_idx:
                self.vocab.token_to_idx[' '] = len(self.vocab.token_to_idx)
                self.vocab.idx_to_token[len(self.vocab.token_to_idx) - 1] = ' '

            # Заполняем паддингами до максимальной длины в пакете
            padded_surnames = pad_sequence(surnames, batch_first=True, padding_value=self.vocab.token_to_idx[' '])

            return {'surname': padded_surnames, 'nationality': torch.tensor(nationalities)}

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        surname = self.X.iloc[idx]
        nationality = self.y.iloc[idx]
        vectorized_surname = self.vectorize(surname)
        return {'surname': vectorized_surname, 'nationality': nationality}

In [37]:
# Шаг 2.6: Обучение модели
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleClassifier, self).__init__()
        self.embedding = nn.Embedding(input_size, 50)
        self.fc = nn.Linear(50, output_size)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x, dim=1)
        x = self.fc(x)
        return x

In [38]:
# Подготовка данных
input_size = len(Vocab("".join(df['surname'].values)).idx_to_token)
output_size = len(nationalities)

train_dataset = SurnamesDataset(train_df['surname'], train_df['nationality'], Vocab("".join(df['surname'].values)))
test_dataset = SurnamesDataset(test_df['surname'], test_df['nationality'], Vocab("".join(df['surname'].values)))

In [39]:
# Используем collate_fn для обработки пакетов с переменной длиной
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=train_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=test_dataset.collate_fn)


In [40]:
# Обучение модели
model = SimpleClassifier(input_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, targets = batch['surname'], batch['nationality']
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [41]:
# Шаг 2.7: Измерение точности на тестовой выборке
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        inputs, targets = batch['surname'], batch['nationality']
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 51.87%


## 3. Классификация обзоров ресторанов

Датасет: https://disk.yandex.ru/d/nY1o70JtAuYa8g

3.1 Считать файл `yelp/raw_train.csv`. Оставить от исходного датасета 10% строчек.

3.2 Воспользоваться функцией `preprocess_text` из 1.1 для обработки текста отзыва. Закодировать рейтинг числами, начиная с 0.

3.3 Разбить датасет на обучающую и тестовую выборку

3.4 Реализовать класс `Vocab` (токен = слово)

3.5 Реализовать класс `ReviewDataset`

3.6 Обучить классификатор

3.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)


In [ ]:
class Vocab:
  def __init__(self, data):
    self.idx_to_token = ...
    self.token_to_idx = ...
    self.vocab_len = ...

In [ ]:
class ReviewDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, review):
    '''Генерирует представление отзыва review при помощи бинарного кодирования (см. 1.2)'''

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return ...